In [2]:
import os
cwd = os.getcwd().replace('/pandas_research', '')
os.chdir(cwd)

In [4]:
import pickle
from dataset import Dataset
import numpy as np
import pandas as pd
import vectorbt as vbt
import pandas_ta as ta
from analyse_candle import create_features


In [5]:
with open('backtest_charts/results.pickle', 'rb') as f:
    result = pickle.load(f)

In [7]:
best_res = {}
for k, v in result.items():
    if not len(v.keys()):
        continue
    best_key = max(v, key=lambda k: v[k]['pf_stat']['Profit Factor'])
    best_res[k] = {}
    best_res[k][best_key] = v[best_key]


In [12]:
data_binance = Dataset().get_data(days=360, ticker='BTCUSDT', ts='1h')
data = data_binance.copy()

In [13]:
data, candles = create_features(data)
data = data.reset_index()

In [20]:
data['entries'] = False
data['exits'] = False
data['short_entries'] = False
data['short_exits'] = False
for i, v in data.iterrows():
    for key, value in best_res.items():
        if v[key] == 0:
            continue
        if list(value.values())[0]['df_stat']['cuts_price'] != v['cuts_price']:
            continue
        if list(value.values())[0]['df_stat'][key] != v[key]:
            continue

        hold_time = int(list(value.keys())[0].split('_')[-1])

        if i + hold_time > len(data) - 1:
            continue
        position = list(value.values())[0]['position']
        if position == 'long':
            data.loc[i, 'entries'] = True
            data.loc[i + hold_time, 'exits'] = True
        if position == 'short':
            data.loc[i, 'short_entries'] = True
            data.loc[i + hold_time, 'short_exits'] = True


In [24]:
pf = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=data['entries'],
    exits=data['exits'],
    short_entries=data['short_entries'],
    short_exits=data['short_exits'],
    fees=0.0006
)
pf.plot().show()

In [25]:
pf.stats()

/home/serg/CRYPTO/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/home/serg/CRYPTO/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/home/serg/CRYPTO/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/home/serg/CRYPTO/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                            0.000000
End                           8640.000000
Period                        8641.000000
Start Value                    100.000000
End Value                      228.635758
Total Return [%]               128.635758
Benchmark Return [%]            -8.705318
Max Gross Exposure [%]         100.000000
Total Fees Paid                145.323424
Max Drawdown [%]                31.350445
Max Drawdown Duration         2530.000000
Total Trades                   878.000000
Total Closed Trades            878.000000
Total Open Trades                0.000000
Open Trade PnL                   0.000000
Win Rate [%]                    49.658314
Best Trade [%]                  10.442863
Worst Trade [%]                 -9.307513
Avg Winning Trade [%]            1.429693
Avg Losing Trade [%]            -1.186776
Avg Winning Trade Duration       6.098624
Avg Losing Trade Duration        5.052036
Profit Factor                    1.184155
Expectancy                       0